In [1]:
import pandas as pd

In [11]:
t5_df = pd.read_csv('~/Downloads/flan-t5-xl.csv')

In [20]:
full_df = pd.read_csv('../data/fact_checking_full_input_information_3_21_23.csv', index_col=[0])

In [15]:
t5_df.rename(columns={'Unnamed: 0': 'dataset_id'}, inplace=True)

In [16]:
t5_df

,dataset_id,differences,facts
0,10806,-0.876394,Tupungato is a part of the continent of <extra...
1,9588,-0.830750,George V Coast is a part of the continent of <...
2,12646,-0.812160,Onyx River is a part of the continent of <extr...
3,2052,-0.790335,The official language of Russia is <extra_id_0...
4,11470,-0.789551,Villa des Roses was created in the country of ...
...,...,...,...
95,10424,-0.489119,The Betty Hutton Show premiered on <extra_id_0...
96,9859,-0.488525,David Beckham is a professional <extra_id_0>. ...
97,11439,-0.485204,"In Kitee, an official language is <extra_id_0>..."
98,13510,-0.483032,Payer Mountains belongs to the continent of <e...


In [28]:
full_df.head()

,relation,subject,object,stem,true,false,dataset_id
0,P47,Norfolk,Suffolk,Norfolk borders with,Suffolk,"['Anabar', 'Vadena', 'Bologna', 'Upper Macungi...",calinet_1
1,P47,Jordan,Israel,Jordan shares a common border with,Israel,"['Mpumalanga', 'Simbach']",calinet_2
2,P47,Kenya,Ethiopia,Kenya shares border with,Ethiopia,"['James City County, Virginia', 'Giridih', 'Ro...",calinet_3
3,P47,Egypt,Israel,Egypt shares its border with,Israel,"['Le Havre', 'Sestriere', 'Nitra District', 'M...",calinet_4
4,P47,Tanzania,Uganda,Tanzania borders with,Uganda,"['La Pampa', 'Ukrainians', 'Ziburu', 'First Cz...",calinet_5


In [54]:
def get_t5_stem(t5_fact):
    extra_id_start_index = t5_fact.index(' <extra_')
    assert(extra_id_start_index != -1)
    return t5_fact[:extra_id_start_index]

In [69]:
def get_dataset_id(t5_fact, df_stems, full_df):
    t5_stem = get_t5_stem(t5_fact).strip()
    # print(f"t5 fact - {t5_fact}")
    # print(f"t5 stem - {t5_stem}")
    for s in df_stems:
        if t5_stem == s:
            return list(full_df[full_df['stem']==s].dataset_id)[0]
    # fail case for stuff we've already gotten rid of 
    # print(f"Could not find original in dataset for {t5_fact}")
    return -1

In [79]:
wrong_facts = []
wrong_ids = []
model = []
resolution = []
for t5_fact in list(t5_df['facts']):
    original_dataset_id = get_dataset_id(t5_fact, list(full_df['stem']), full_df)
    
    if original_dataset_id == -1:
        continue
        
    row = full_df[full_df['dataset_id'] == original_dataset_id]
    wrong_facts.append(list(row['stem'])[0] + " " + list(row['false'])[0] + " " + list(row['true'])[0])
    wrong_ids.append(original_dataset_id)
    model.append("t5-xl")
    resolution.append("todo")

In [80]:
error_df_subset = pd.DataFrame({'model': model, 'dataset_id': wrong_ids, 'resolution': resolution, 'wrong_fact_with_true_false': wrong_facts})

In [81]:
error_df_subset.head()

,model,dataset_id,resolution,wrong_fact_with_true_false
0,t5-xl,rome_6633,todo,Tupungato is a part of the continent of ['Afri...
1,t5-xl,rome_82,todo,George V Coast is a part of the continent of [...
2,t5-xl,rome_16432,todo,Onyx River is a part of the continent of ['Afr...
3,t5-xl,calinet_3198,todo,The official language of Russia is ['collabora...
4,t5-xl,rome_10178,todo,Villa des Roses was created in the country of ...


In [82]:
error_df_subset.to_csv('~/Downloads/t5_errors.csv', index=False)